In [18]:
import matplotlib.pyplot as plt  
# iscrtavanje slika i plotova unutar samog browsera
%matplotlib inline 

import matplotlib.pylab as pylab
# prikaz vecih slika 
pylab.rcParams['figure.figsize'] = 21,15

import numpy as np
import cv2 # OpenCV biblioteka

def show_in_window_and_below(img, below=True):
    if (below):
        plt.imshow(img, 'gray')
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# current image extension
ext = '.JPG'

BLACK = 0 # black pixel's color
WHITE = 255 # white pixel's color

In [19]:
# runs calculations
def calculate_runs(img):
    runs = [[1] for x in xrange(img.shape[1])] # each column starts with 1 black pixel
    black_runs_flat, white_runs_flat = [], []
    for col in xrange(img.shape[1]): # iterate through all columns
        img[0,col] = 0 # PAINT THE FIRST PIXEL IN PREDEFINED COLOR, to make all columns start the same
        run_index = 0 # start the run
        for row in xrange(1, img.shape[0]): # for each pixel/row in current column
            if (img[row-1 ,col] != img[row, col]):  # if they are not the same,
                # memorize the old run in corresponding array
                if (run_index % 2 == 0): # black run
                    black_runs_flat.append(runs[col][run_index])
                else:
                    white_runs_flat.append(runs[col][run_index])
                # start a new run
                run_index += 1
                runs[col].append(0)
            runs[col][run_index] += 1     # add a pixel to the current run
        # the column ended ==> save the last run for the ended column
        if (run_index % 2 == 0): # black run
            black_runs_flat.append(runs[col][run_index])
        else:
            white_runs_flat.append(runs[col][run_index])
    return runs, black_runs_flat, white_runs_flat

### Line *thickness* and *spacings* (black and white runs) analysis

In [20]:
from collections import Counter

def calculate_line_thickness(black_flat):
    num_top = 4 # number of top/most common runs
    black_count = Counter(black_flat) # Counter({1: 3, 8: 1, 3: 1, ...})
    m_c_black = black_count.most_common(num_top)
    print 'Top', num_top, 'most_common_black_runs:', m_c_black
    
    m_c_black1 = m_c_black[0][0]
    if (len(m_c_black) >= 2):
        m_c_black2 = m_c_black[1][0]
        if (m_c_black1*3 < m_c_black2): # kind of a sanity check
            line_thickness = m_c_black1
        else:
            line_thickness = (m_c_black1 + m_c_black2) / 2.
    else:
        line_thickness = m_c_black1
    
    print '>>> Calculated line thickness:  ', line_thickness
    return line_thickness

def calculate_line_spacing(white_flat, image_height):
    num_top = 4 # number of top/most common runs
    white_count = Counter(white_flat) # print white_count.most_common(50)
    m_c_white = white_count.most_common(num_top)
    print 'Top', num_top, 'most common white runs', m_c_white
    
    m_c_white1 = m_c_white[0][0]
    if (len(m_c_white) >= 2):
        m_c_white2 = m_c_white[1][0]
        
        if (m_c_white1 > image_height*0.5): # sanity check
            line_spacing = m_c_white2
        else:
            if (m_c_white2 > image_height*0.5):
                line_spacing = m_c_white1
            else:
                line_spacing = (m_c_white1 + m_c_white2) / 2.
    else:
        line_spacing = m_c_white[0][0]
    print '>>> Calculated line spacing: ', line_spacing
    return line_spacing

In [21]:
def remove_staff_lines(img, runs, line_height, staff_thickness_multiplier):
    # copy the image.. python is pass-by-object-reference so it is necessary!
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    #edit the image
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            if (r % 2 == 0): # black runs # every black run longer than 2 * line_height is deleted/whitened
                if (run_length < line_height * staff_thickness_multiplier):
                    # ++ AKO JE SLJEDEĆI/PRETHODNI %% BIJELI %% RUN = VISINA PRAZNINE +-1
                    # ++ AKO JE SLJEDEĆI/PRETHODNI %%  CRNI  %% RUN = VISINA LINIJE +-1
                    p[cumulative:cumulative + run_length, c] = [255]*(run_length)
            #else: # white runs
            #    do something maybe ?
            cumulative += run_length
    return p

In [22]:
def remove_staff_lines_with_lines_only(img, runs, line_height, staff_thickness_multiplier, lines_only_img):
    # copy the image.. python is pass-by-object-reference so it is necessary!
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    #edit the image
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            if (r % 2 == 0): # black runs # every black run longer than 2 * line_height is erased
                if (run_length < line_height * staff_thickness_multiplier):
                    for compare_px in xrange(cumulative, cumulative + run_length):
                        if (lines_only_img[compare_px, c] == BLACK):
                            p[compare_px, c] = 255
                        # p[cumulative:cumulative + run_length, c] = [255]*(run_length)
            cumulative += run_length
    return p

In [23]:
#########
# USE:
#    = in lines-only image, to LOCATE the lines AND/OR check if there is a line on the current location/run
#        - easier to find lines, since there are no other elements
#        - takes more time, since we need to generate the lines-only image,
#           ,BUT THAT IS NOT A PROBLEM SINCE WE WILL NEED IT TO LOCATE THE LINES
#
#    = in binary image, to check if there is a line on the current location/run
#
# da se utvrde linije treba samo odrediti visinu, a ne 
# TIP: mozda za svaki linijski sistem (ili cak liniju) cuvati vise x, koordinata,
#   recimo na svaku petinu sirine slike provjeravati lokacije
#   linijskih sistema (ili pojedinacnih linija) na vise mijesta u slici:
#       |       |        |      |
#       V       V        V      V
# -----..____..--------------------  <== curved line, others are ok
# --------------------------------- 
# ---------------------------------
# ---------------------------------
# ---------------------------------
######
# x je niz od onoliko crnih piksela koliko je prosjecna debljina linije (+-1 ili 2)
# x = [0] * (int(thickness)-1)
# y je niz od onoliko crnih piksela koliko je prosjecna debljina linije + 2 ili 3 ||| INT!
#   ~ pikseli iz y niza pocinju od posljednje tacke posmatrane linije (tj.kandidata za liniju)
# y = current_pos + int(spacing * 0.9 ili 0.8)
# print x_in_y(x, y)
###
def x_in_y(x, y):
#     print 'checking', x, 'in', y
    try:
        x_len = len(x)
    except TypeError:
        x_len = 1
        x = type(y)((x,))

    for i in xrange(len(y)):
        if (y[i : i+x_len] == x):
            return True
    return False

# a = [0,0,1,1,1,0,0,0,0]
# b = [1,1,1]
# b = 0 # works also
# print x_in_y(b, a)

In [24]:
# # # # # # # #
# SKIP THIS !!!  This was waaay back then... it's old testing
# # # # #
def parse_image(img, threshold_type, block_size, c_value, staff_thickness_multiplier, params=[]):
    t_t, b_s, s_t_m = threshold_type, block_size, staff_thickness_multiplier
    print('=========\nthreshold_type: {}, block_size: {}, c_value: {}, staff_thickness_multiplier: {}'.format(t_t, b_s, c_value, s_t_m))
    img_ada = cv2.adaptiveThreshold(img, 255, threshold_type, cv2.THRESH_BINARY, block_size, c_value)
    
    dilate_kernel = np.ones((1,30), dtype=np.int) # np.ones((kernel_w, kernel_h), dtype=np.int);
    # staff lines LOCATIONS, along with lines-only image
    lines_only_img, locations = cv2.dilate(img_ada, kernel, iterations=1)
    
    # runs calculation
    runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada)
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # needs image height
    result = rm_staff_lines_up_down_neighbours(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier)
#     distance = int(line_spacing * 0.5)
#     result = rm_staff_lines_side_neighbours(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier, distance)
#     result = rm_s(img_ada, runs, line_thickness, line_spacing, staff_thickness_multiplier, distance)
    
    cv2.imwrite('./images/dataset/run_X/params_'+str(t_t)+'_'+str(b_s)+'_'+str(c_value)+'_'+str(s_t_m)+ext, result)
    #cv2.imwrite('./images/dataset/run_Y/params_ORIGINAL.jpg', img_ada)

# img = cv2.imread('images/dataset/muzikanti'+ext, 0) #  0 -->  read as grayscale
# parse_image(img, cv2.ADAPTIVE_THRESH_MEAN_C, 33, 35, 1.5)

In [26]:
# # # # # # # #
# SKIP THIS !!!  This was waaay back then... it's old testing
# # # # #
# Test various kernel dimensions on  binarized images with
# morph. operations: erosion, dilation, opening  and  closing
if (False): # kekeke
    kernel_widths = [1, 2]
    kernel_heights = [7, 11, 15]

    block_sizes = [51, 51]
    c_values = [35, 45]
    ada_method = cv2.ADAPTIVE_THRESH_MEAN_C
    thresh = cv2.THRESH_BINARY
    for i in range(1):
        img_path = './images/dataset/viva{}_up.jpg'.format(i+1)
        img = cv2.imread(img_path, 0) # grayscale
        img = cv2.adaptiveThreshold(img, 255, ada_method, thresh, block_sizes[i], c_values[i])
        for kernel_width in kernel_widths:
            for kernel_height in kernel_heights:
                #kernel_width = kernel_height # ONLY FOR TESTING!!!
                kernel = np.ones((kernel_width, kernel_height), dtype=np.int)
                eroded = cv2.erode(img, kernel, iterations=1)
                dilated = cv2.dilate(img, kernel, iterations=1)
                er_b4_dil = cv2.dilate(eroded, kernel, iterations=1)
    #             dil_b4_er = cv2.erode(dilated, kernel, iterations=1) # useles...

                cv2.imwrite('./images/kernel_2/ER_b_{}_c_{}_kw_{}_kh_{}.jpg'\
                            .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  eroded)
    #             cv2.imwrite('./images/kernel_2/DIL_b_{}_c_{}_kw_{}_kh_{}.jpg'\
    #                         .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  dilated) # no need to save dil.
                cv2.imwrite('./images/kernel_2/ER_B4_DIL_b_{}_c_{}_kw_{}_kh_{}.jpg'\
                            .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  er_b4_dil)
    #             cv2.imwrite('./images/kernel_2/DIL_B4_ER_b_{}_c_{}_kw_{}_kh_{}.jpg'\
    #                         .format(block_sizes[i], c_values[i], kernel_width, kernel_height),  dil_b4_er)
    # end

In [33]:
# Use all three algorithms on the input image
#   1 - simple:  with no parameters (with optional erosion before the alg.)
#   2 - up-down: basic call, + 3 calls with dilation (kernel_1 1 and 2 iterations, kernel_2)
#   3 - side:    with two distances: line_spacing*A and line_spacing*B. A and B are from [0.1, 2.0]
block_sizes = [51]#, 51] #  ]#
c_values = [45]#, 45]    #  ]#
thresh = cv2.ADAPTIVE_THRESH_MEAN_C
method = cv2.THRESH_BINARY
# img = cv2.imread('images/dataset/muzikanti'+ext, 0) #  0 => read as grayscale
img = cv2.imread('./images/dataset/viva1_up.jpg', 0)

for i in range(0): # range(len(block_sizes))
    block = block_sizes[i]
    c = c_values[i]
    img = cv2.adaptiveThreshold(img, 255, thresh, method, block, c)
    path_regular = './images/e_viva/b_{}_c_{}_LINES.jpg'
    cv2.imwrite(path_regular.format(block, c), img)
    # PREPARE image for lines detection  --  ER_B4_DIL_b_51_c_35_kw_2_kh_7
    kernel = np.ones((2, 7), dtype=np.int)
    eroded = cv2.erode(img, kernel)
    er_pa_dil = cv2.dilate(eroded, kernel)
    path_er_pa_dil = './images/e_viva/b_{}_c_{}_ER_B4_DIL_k_2x7.jpg'
    cv2.imwrite(path_er_pa_dil.format(block, c), er_pa_dil)
    img = er_pa_dil
    
    runs, black_runs_flat, white_runs_flat = calculate_runs(img) # runs calculation, for thickness and spacing
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img.shape[0]) # needs image height
    
    thickness_mul = 1.5 # staff_thickness_multiplier
    
    # Remove staff lines - RUNS algorithm ~~Simplest~~ (Aleksandar's)
    rm_s_l = remove_staff_lines(img, runs, line_thickness, thickness_mul)
#     path_regular = './images/e_viva/ER_{}_{}_b_{}_c_{}_RM_S_L.jpg'
#     cv2.imwrite(path_regular.format(k_w, k_h, block, c), rm_s_l)
    path_regular = './images/e_viva/b_{}_c_{}_RM_S_L.jpg'
    cv2.imwrite(path_regular.format(block, c), rm_s_l)
    
    # Remove staff lines - UP-DOWN algorithm (Aleksandar's)
    rm_s_l_up_down = rm_staff_lines_up_down_neighbours(img, runs, line_thickness, line_spacing, thickness_mul)
    path_up_down = './images/e_viva/b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down.format(block, c), rm_s_l_up_down)
    
    up_down_dilated = cv2.dilate(rm_s_l_up_down, kernel)
    path_up_down_dil = './images/e_viva/DIL_{}_{}_b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down_dil.format(k_w, k_h, block, c), up_down_dilated)
    
    up_down_dilated_x2 = cv2.dilate(rm_s_l_up_down, np.ones((3, 1), dtype=np.int), iterations=2)
    path_up_down_dil_x2 = './images/e_viva/DIL_3_1_TWO_iters_b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down_dil_x2.format(block, c), up_down_dilated_x2)
    
    k_w, k_h = 5, 2 # above kernel is 5 1. This is 5 2
    kernel = np.ones((k_w, k_h), dtype=np.int)
    up_down_dilated = cv2.dilate(rm_s_l_up_down, kernel)
    path_up_down_dil = './images/e_viva/DIL_{}_{}_b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
    cv2.imwrite(path_up_down_dil.format(k_w, k_h, block, c), up_down_dilated)
    
    # Remove staff lines - SIDE algorithm (Filip's)
    distance = int(line_spacing * 0.8) ### check for: * 1, * 1.3, * 1.5
    rm_s_l_side = rm_staff_lines_side_neighbours(img, runs, line_thickness, line_spacing, thickness_mul, distance)
    path_side = './images/e_viva/b_{}_c_{}_RM_S_L_SIDE_dist_{}.jpg'
    cv2.imwrite(path_side.format(block, c, distance), rm_s_l_side)
    
    distance = int(line_spacing * 1) ### check for: * 1, * 1.3, * 1.5
    rm_s_l_side = rm_staff_lines_side_neighbours(img, runs, line_thickness, line_spacing, thickness_mul, distance)
    path_side = './images/e_viva/b_{}_c_{}_RM_S_L_SIDE_dist_{}.jpg'
    cv2.imwrite(path_side.format(block, c, distance), rm_s_l_side)
# end


< # # # #  # # # #  # # # #  # # # #  # # # #  # # # >
### (RLE) run-based approach to  LOCATING  STAVES
< # # # #  # # # #  # # # #  # # # #  # # # #  # # # >

In [34]:
# Adds "help" lines and spaces (eng. "ledger lines/spaces") to `locaions` dictionary
def add_helper_spaces_and_lines(locations, line_thickness, line_spacing, cumulative):
    line_thickness = int(round(line_thickness)) # round(2.4) = 2 ||| round(2.6) = 3  :)
    line_spacing = int(round(line_spacing))
    
    help_space_3_up_y = cumulative - (2 * line_thickness) - (3 * line_spacing) 
    locations[help_space_3_up_y] = (0, line_spacing)
    help_line_2_up_y = cumulative - (2 * line_thickness) - (2 * line_spacing)
    locations[help_line_2_up_y] = (1, line_thickness)
    help_space_2_up_y = cumulative - (1 * line_thickness) - (2 * line_spacing)
    locations[help_space_2_up_y] = (2, line_spacing)
    help_line_1_up_y = cumulative - (1 * line_thickness) - (1 * line_spacing)
    locations[help_line_1_up_y] = (3, line_thickness)
    help_space_1_up_y = cumulative - (0 * line_thickness) - (1 * line_spacing)
    locations[help_space_1_up_y] = (4, line_spacing)
    
    lowest_line = max(locations.keys()) # MAX because the lower the LINE, the bigger the Y-coordinate
    lowest_line_end = lowest_line + locations[lowest_line][1]
    
    help_space_1_down_y = lowest_line_end + (0 * line_thickness) + (0 * line_spacing)
    locations[help_space_1_down_y] = (14, line_spacing)
    help_line_1_down_y = lowest_line_end + (0 * line_thickness) + (1 * line_spacing)
    locations[help_line_1_down_y] = (15, line_thickness)
    help_space_2_down_y = lowest_line_end + (1 * line_thickness) + (1 * line_spacing)
    locations[help_space_2_down_y] = (16, line_spacing)
    help_line_2_down_y = lowest_line_end + (1 * line_thickness) + (2 * line_spacing)
    locations[help_line_2_down_y] = (17, line_thickness)
    help_space_3_down_y = lowest_line_end + (2 * line_thickness) + (2 * line_spacing) 
    locations[help_space_3_down_y] = (18, line_spacing)
    return 0 # everything is OK

def run_is_spacing_candidate(run_length, line_spacing):
    return (line_spacing * 0.85 <= run_length <= line_spacing * 1.15)

def run_is_line_candidate(run_length, line_thickness):
#     lower_bound = line_thickness * 0.4 if line_thickness > 3.6 else 1 # Helps with thin average lines
    return (line_thickness * 0.4 <= run_length <= line_thickness * 1.6)
    
# Checks if there is a staff and returns a tuple (found, start_pixel, locations, end_pixel)
#   1.1) True,  or  False
#   1.2) The y-coordinate of the stave's START (start_pixel),  or  -1
#   1.3) Dictionary of pairs: (line_or_space_start_y, (line_space_CODE, line_thickness))
#   1.4) The y-coordinate of the stave's END (end_pixel),  or  -1
def get_staff_with_spaces(run_index, runs, image_column, cumulative, line_thickness, line_spacing):
#     print ' ~~ get_staff_with_spaces ~~\n  checking', image_column[cumulative:cumulative+150]
    print ' ~~ get_staff_with_spaces ~~\n  checking:' #, image_column#[:cumulative], image_column[cumulative:]
    print '\t run_index,', run_index, '/', len(runs), '\t runs:', runs
    # initialize the return dictionary with a line number 1, which has the CODE=5. Why 5? See the next line
    # >> help_space3_up (CODE=0), help_line2_up, help_space2_up, help_line1_up, help_space1_up, FIRST_LINE
    locations = {cumulative: (5, runs[run_index])} # first line starts at the current index
    line_space_counter = 6 # first line is saved, now go for other SPACES AND LINES
    # cumulative -> how low (Y) are we in this image column - Make_first_row_black_in_runs_calc idea will be discarded!
    end_pixel = cumulative + runs[run_index] # determines the y-coordinate where this the last-found element end
    stave_found = True
    
    for i in range(1, 9):
        run_length = runs[run_index + i]
        if i%2 == 1: # i is odd for line_spacings, since for i=0 we get first LINE's index (run_index)
            if not run_is_spacing_candidate(run_length, line_spacing):
                stave_found = False
                break
        else: # i%2 == 0
            if not run_is_line_candidate(run_length, line_thickness):
                stave_found = False
                break
        # we came to execute this line, so this run (space/line) is ok
        locations[end_pixel] = (line_space_counter, run_length) # add the SPACE/LINE
        line_space_counter += 1 # look for the next SPACES AND LINES
        end_pixel += run_length # advance down the column
    
    if not stave_found:
        return False, -1, {}, -1 # not_found, no_start, no_locations, no_end
    
    # prepare the return values
    add_helper_spaces_and_lines(locations, line_thickness, line_spacing, cumulative)
    start_pixel = min(locations.keys())
    # get the last line's y_coordinate and add her height to it -- that's where this stave ends
    lowest_space = max(locations.keys()) # MAX because the lower the SPACE, the bigger the Y-coordinate
    end_pixel = lowest_space + locations[lowest_space][1] # lowest SPACE at `lowest_space`, height: locations[lowest_space][1]
    return True, start_pixel, locations, end_pixel

# IS beginning of the first stave close to the beginning of the second?
def staves_are_close(stave1, stave2, line_thickness):
    return abs(stave1[1][0] - stave2[1][0]) < line_thickness

def find_staves_in_runs(runs, image_column, line_thickness, line_spacing):
    run_index = 0 # current run index
    cumulative = 0 # how many pixels did we pass - for getting run's color# The paint_the_first_row_black idea will be discarded!!!
    
    staff_counter = 0
    staves = {}
    while (run_index < len(runs)):# WHILE, because we need more control over the index vlue
        run_length = runs[run_index]
#         print 'cumulative is now:',cumulative,'| runs[',run_index,'] =',run_length,'| len(runs):',len(runs)
        if (image_column[cumulative] == BLACK and run_is_line_candidate(run_length, line_thickness)):
#             print 'checking a BLACK run of size', run_length
            staff_is_found, start_pixel, staff, end_pixel = get_staff_with_spaces(run_index, runs, image_column, cumulative, line_thickness, line_spacing)
            if (staff_is_found): # YEA!
                print '\n<<<< we got them linez! >>>'
                staff_counter += 1
                staves[start_pixel] = (staff_counter, staff, end_pixel)
                # A staff was found, so we need to go down 8 runs (4*spacing, 4*line) until the last line of this staff
                for r_i in range(run_index, run_index+9): # OR 8??
                    cumulative += runs[r_i] # print 'adding', runs[r_i], 'to cumulative'
                run_index += 8
            else: # staff was NOT found
                cumulative += run_length
        else:
            cumulative += run_length
        run_index += 1
    
    return len(staves), staves # return the size and the staves dictionary

# check that NO staves from the `staves` dictionary are close to this `stave`
def no_staves_are_close(staves, stave, staff_start):
#     close_range = int(line_thickness * 1.5 + 1) / 2
    print 'close_range = (', stave[2], '-', staff_start,') / 2'
    close_range = (stave[2] - staff_start)/2
    starts_list = range(staff_start - close_range, staff_start + close_range + 1) # + 1 because range() doesn't include the last element
    starts_list.remove(staff_start)
#     starts_list = [staff_start - 2, staff_start - 1, staff_start + 1, staff_start + 2] #%% S I M P L E S T  :D  %%%%
    intersection = [start for start in starts_list if start in staves]
    return intersection == [] # Return value can "easily" be changed to list of matches
    # EXAMPLE: return [-3, 1, 5] if there are staves beginning 3px ABOVE, 1px BELOW, or 5px BELOW this staff
    # OR: return a list of staves that are close: [staves[start] for start in starts_list if start in staves]

def locate_lines_with_runs(lines_only_img):
    print 'lines_only_img.shape', lines_only_img.shape
    img_height, img_width = lines_only_img.shape # unpack values
    
    # runs calculation, for thickness and spacing
    runs, black_runs_flat, white_runs_flat = calculate_runs(lines_only_img)
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_height)
    
    # get the columns that we will check for lines
    # columns = [img_width/5, img_width/5+1, img_width/5+2, img_width/3, img_width/3+1, img_width/3+2]
    columns_to_check = [img_width/5, img_width/4, img_width/3, img_width/2, img_width/3*2, img_width/4*3]
    # w/5, w/4, w/3, w/2, w/3*2, w/4*3 # for a 2400px wide image, values are: 480 600 800 1200 1600 1800
    print 'columns_to_check', columns_to_check
    
    search_width = 3 # how wide will we look arround each pixel - to BOTH sides!
    resulting_staves = {} # DICTIONARY!
    
    for col_to_check in columns_to_check:
        print '\n -- -- -- ~~ -- -- ~~ -- ~~ -- ~~\nchecking around column', col_to_check
        start_column = col_to_check - search_width
        end_column = col_to_check + search_width
        column_range = range(start_column, end_column + 1) # + 1 because range() doesn't include the last element
        print 'range(start_column, end_column)', column_range
        for col in column_range:
            print '\n ~~~~~~~~\n  checking column >>>', col, '<<<'
            image_column = lines_only_img[0:img_height, col].tolist() # convert numpy array to python list
            num_of_staves, staves = find_staves_in_runs(runs[col], image_column, line_thickness, line_spacing)
            if (num_of_staves > 0):
                for staff_start in staves: # iterate through keys # or something like that
                    # IF there is NO staffs that are close to this one (on y-axis)
                    if (no_staves_are_close(resulting_staves, staves[staff_start], staff_start)):
                        resulting_staves[staff_start] = staves[staff_start]
                break # !!! no need to check arround the `col_to_check` any more - go to next part of the image
    return resulting_staves # , stave_possibilities


In [35]:
def dilate_and_save(img, kernel_w, kernel_h):
    '''Dilates a binary image with kernel of specified dimensions.
    saves the image to hard drive and returns the saved image'''
    kernel = np.ones((kernel_w, kernel_h), dtype=np.int)
    lines_only_img = cv2.dilate(img, kernel, iterations=1)
    lines_only_img_path = './images/locate_lines/dil_{}_{}.jpg'.format(kernel_w, kernel_h)
    cv2.imwrite(lines_only_img_path, lines_only_img)
    return lines_only_img

def erode_and_save(img, kernel_w_e, kernel_h_e, kernel_w_d=0, kernel_h_d=0):
    '''Erodes a binary image with kernel of specified dimensions.
    saves the image to hard drive and returns the saved image'''
    if (kernel_w_d == 0):
        dil_str = ''
    else:
        dil_str = '_dil_{}_{}'.format(kernel_w_d, kernel_h_d)
    kernel = np.ones((kernel_w_e, kernel_h_e), dtype=np.int)
    lines_only_img = cv2.erode(img, kernel, iterations=1)
    lines_only_img_path = './images/locate_lines/er_{}_{}{}.jpg'.format(kernel_w_e, kernel_h_e, dil_str)
    cv2.imwrite(lines_only_img_path, lines_only_img)
    return lines_only_img

### Two approaches on getting the lines-only image:
####  |__ 1: dilate and then erode with a very wide kernel (written right below this heading)
####  |__ 2: use the previous approach's image + remove all non-staff-line symbols (implementation started in 'testing stuff' notebook)

In [74]:
def get_lines_only_img(img):
    threshold = cv2.ADAPTIVE_THRESH_MEAN_C
    method, block, c = cv2.THRESH_BINARY, 55, 9 # 35, 13  || 45, 11
    img_ada = cv2.adaptiveThreshold(img, 255, threshold, method, block, c)

    # dilate with 1x50 --> erode with 2x50  | this has the best results
    kernel_w, kernel_h = 1, 50
    dilated = dilate_and_save(img_ada, kernel_w, kernel_h)
    kernel_w_e, kernel_h_e = 2, 50
    return erode_and_save(dilated, kernel_w_e, kernel_h_e, kernel_w, kernel_h)

def print_staff_locations(staff_locations):
    staves_keylist = staff_locations.keys()
    staves_keylist.sort()
    print '\nSORT %d staves and their lines by y-coordinates' % (len(staff_locations))
    # print staff_locations
    for staves_key in staves_keylist:
        print "STAVE AT Y = %s" % (staves_key)
        staff = staff_locations[staves_key][1]
        staff_keylist = staff.keys()
        staff_keylist.sort()
        for staff_key in staff_keylist:
            print '\tstaff element %s: %s' % (staff_key, staff[staff_key])
        print ''
    return 0

def get_line_locations(img, print_result=True):
    '''Locate lines on the given image and print results (if print_results is `True`)
    Returns: staff lines LOCATIONS and the lines-only image'''
    lines_only_img = get_lines_only_img(img) # get_lines_only_img_CLEAR # approach 2 - maybe in the future
    
    # use the lines-only image to locate the staves
    staff_locations = locate_lines_with_runs(lines_only_img)
    
    if (print_result):
        print_staff_locations(staff_locations)
    return staff_locations
    
img = cv2.imread('./images/dataset/newest/bolujem.jpg', 0) # 0 --> read as grayscale
staf_locs = get_line_locations(img)

lines_only_img.shape (3264L, 2448L)
Top 4 most_common_black_runs: [(5, 25554), (6, 5450), (4, 3868), (1, 2448)]
>>> Calculated line thickness:   5.5
Top 4 most common white runs [(16, 19971), (17, 4490), (15, 3210), (313, 561)]
>>> Calculated line spacing:  16.5
columns_to_check [489L, 612L, 816L, 1224L, 1632L, 1836L]

 -- -- -- ~~ -- -- ~~ -- ~~ -- ~~
checking around column 489
range(start_column, end_column) [486, 487, 488, 489, 490, 491, 492]

 ~~~~~~~~
  checking column >>> 486 <<<
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 2 / 32 	 runs: [1, 503, 6, 15, 5, 16, 5, 16, 5, 16, 6, 309, 5, 16, 5, 16, 5, 16, 5, 16, 5, 306, 5, 16, 4, 17, 4, 16, 5, 15, 6, 1878]

<<<< we got them linez! >>>
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 12 / 32 	 runs: [1, 503, 6, 15, 5, 16, 5, 16, 5, 16, 6, 309, 5, 16, 5, 16, 5, 16, 5, 16, 5, 306, 5, 16, 4, 17, 4, 16, 5, 15, 6, 1878]

<<<< we got them linez! >>>
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 22 / 32 	 runs: [1, 50


<<<<  Pitch recognition  >>>>
---

### Scientific pitch notation is used in MusicXML (see <a href="https://en.wikipedia.org/wiki/Scientific_pitch_notation"> wikipedia page about Scientific pitch notation</a>)

In [75]:
# We currently support notes
# from: above 2nd UPPER ledger/helper line
# to:   below 2nd LOWER ledger/helper line
clefs = {
  'treble': ['D6','C6',  'B5','A5','G5','F5','E5','D5','C5',  'B4','A4','G4','F4','E4','D4','C4',  'B3','A3','G3'],
  'bass': ['F4','E4','D4','C4',  'B3','A3','G3','F3','E3','D3','C3',  'B2','A2','G2','F2','E2','D2','C2',  'B1']
}

In [76]:
# get the staff that contains the sent symbol (his y coordinate, actually)
# returns the matched staff's key from staff_locations dictionary
def get_staff(symbol_y_coordinate, staff_locations):
    # get staff locations keys (y-coordinates of staves beginnings)
    sorted_staff_locations_keys = staff_locations.keys()
    sorted_staff_locations_keys.sort() # make them sorted # NOTE :::: sorted_keys.reverse()  REVERSES THE SORT! (returns None!)
    
    # go through sorted keys in dictionary of staves (they are y-coordinates of each staff's beginning)
    # and choose only ones that are above the symbol's y-coordinate (i.e. their y-coordinate is a smaller number)
    candidates = [staff_key for staff_key in sorted_staff_locations_keys if (staff_key <= symbol_y_coordinate)]
    
#     print candidates, '| size:', len(candidates)
#     print '  smallest difference is between the lastly added candidate'
    chosen_staff_key = candidates[len(candidates) - 1]
#     print '  best match is:', chosen_staff_key
    
    # check that the symbol is valid, i.e. not lower than the chones staff's height
    # (exaple: if staff starts at 500px and ends at 700px and the symbol_y = 800px)
    if (staff_locations[chosen_staff_key][2] >= symbol_y_coordinate):
        return chosen_staff_key
    # we came here, so the symbol location is not valid (see the two lines above)
    return None

# returns the staff element's CODE: 0 for the 3rd upper spacing above the staff, ..., 5 for the 1st staff line
def get_staff_element_code(symbol_y_coordinate, chosen_staff):
#     print 'symbol_y:', symbol_y_coordinate, 'chosen_staff:', chosen_staff
    staff_elements = chosen_staff[1]
    # get staff-line locations keys (y-coordinates of staff-lines beginnings)
    sorted_chosen_staff_keys = staff_elements.keys() # [1] ==> the 0th tuple element is 
    sorted_chosen_staff_keys.sort() # make them sorted # NOTE :::: sorted_keys.reverse()  REVERSES THE SORT! (returns None!)
    
#     print 'SORTED_chosen_staff_keys:', sorted_chosen_staff_keys
    
    candidates = [staff_elem_key for staff_elem_key in sorted_chosen_staff_keys if (staff_elem_key <= symbol_y_coordinate)]
    print '\n', candidates, '| size:', len(candidates)
#     print '  smallest difference is between the lastly added candidate'
    chosen_line_key = candidates[len(candidates) - 1]
    print '  best match is: staff_elements[%d] = %s' % (chosen_line_key, staff_elements[chosen_line_key])
    return staff_elements[chosen_line_key][0] # 0th tuple element is the CODE, 1st is element's height

def get_pitch(staff_element_code, clef):
    return clefs[clef][staff_element_code]

def get_note(symbol_y_coordinate, staff_locations, clef='treble'):
    staff_key = get_staff(symbol_y_coordinate, staff_locations)
    staff = staff_locations[staff_key]
    staff_element_code = get_staff_element_code(symbol_y_coordinate, staff)
    pitch = get_pitch(staff_element_code, clef)
    print 'found pitch for y = %d  ==> %s' % (symbol_y_coordinate, pitch)
    return pitch


### test pitch-related functions

In [103]:
symbol_y_coordinates = [527, 544, 566] #, 599, 997, 998, 999, 1000]
for symbol_y in symbol_y_coordinates:
    get_note(symbol_y, staf_locs, clef='treble') # `clef` param for clarity :)


[441, 458, 464, 481, 487, 504, 509, 525] | size: 8
  best match is: staff_elements[525] = (7, 5)
found pitch for y = 527  ==> D5

[441, 458, 464, 481, 487, 504, 509, 525, 530] | size: 9
  best match is: staff_elements[530] = (8, 16)
found pitch for y = 544  ==> C5

[441, 458, 464, 481, 487, 504, 509, 525, 530, 546, 551] | size: 11
  best match is: staff_elements[551] = (10, 16)
found pitch for y = 566  ==> A4


In [73]:
def last_test():
#     img = cv2.imread('./images/dataset/sviraj_up.jpg', 0) # read the image as GRAYcscale
    img_name = 'composicion'
    root_path = './images/dataset/' + img_name + '/'
    img = cv2.imread(root_path + img_name + '_up.jpg', 0) # read the image as GRAYcscale
    
    adaptiveMethod, thresholdType, blockSize, C = cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 35
    img_ada = cv2.adaptiveThreshold(img, 255, adaptiveMethod, thresholdType, blockSize, C)
#     img_ada = cv2.erode(img_ada, np.ones((1,7), dtype=np.int)) # thickens the lines
    
    runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada) # runs calculation, for thickness and spacing
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # calculation needs image height
    
    lines_only_img = get_lines_only_img(img) # ...
    # line_thickness = 3 # CHEAT LINE
    erode_width = int(line_thickness+1) # set to be 4 allways?
    lines_only_img = cv2.erode(lines_only_img, np.ones((erode_width, 1), dtype=np.int)) # thickens the lines
    
    thickness_mul = 1.5 # staff_thickness_multiplier
#     thickness_mul = 2 # staff_thickness_multiplier
    
#     we can send the grayscale image, also :D  Just send `img` instead of `img_ada`
    result = remove_staff_lines_with_lines_only(img_ada, runs, line_thickness, thickness_mul, lines_only_img)
    result_path = root_path + 'RM_S_L_LINES_ONLY_b{}_c{}_ada{}_er_kw{}.jpg'.format(blockSize, C, adaptiveMethod, erode_width)
    cv2.imwrite(result_path, result)
    
#     runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada) # runs calculation, for thickness and spacing
#     line_thickness = calculate_line_thickness(black_runs_flat)
#     line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # needs image height
    
    thickness_mul = 1.5 # staff_thickness_multiplier
    
#     # Remove staff lines - RUNS algorithm ~~Simplest~~ (Aleksandar's)
#     rm_s_l = remove_staff_lines(img_ada, runs, line_thickness, thickness_mul)
#     path_regular = root_path + 'b_{}_c_{}_RM_S_L.jpg'
#     cv2.imwrite(path_regular.format(blockSize, C), rm_s_l)
    
#     # Remove staff lines - UP-DOWN algorithm (Aleksandar's)
#     rm_s_l_up_down = rm_staff_lines_up_down_neighbours(img_ada, runs, line_thickness, line_spacing, thickness_mul)
#     path_up_down = root_path + 'b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
#     cv2.imwrite(path_up_down.format(blockSize, C), rm_s_l_up_down)
    
#     # Remove staff lines - SIDE algorithm (Filip's) # -----%%%%%%====%%%%  ADDED  LINES-ONLY-IMAGE %%%%%%%%==%%%==%%-
#     distance = int(line_spacing * 2.5)+1 ### check for: * 1, * 1.3, * 1.5
#     rm_s_l_side = rm_staff_lines_side_neighbours(img_ada, runs, line_thickness, line_spacing, thickness_mul, distance, lines_only_img)
#     path_side = root_path + 'ADA_b_{}_c_{}_RM_S_L_SIDE_dist_{}.jpg'
#     cv2.imwrite(path_side.format(blockSize, C, distance), rm_s_l_side)
    
#     distance = int(line_spacing * 1.7)+1 ### check for: * 1, * 1.3, * 1.5
#     rm_s_l_side = rm_staff_lines_side_neighbours(img_ada, runs, line_thickness, line_spacing, thickness_mul, distance, lines_only_img)
#     path_side = root_path + 'ADA_b_{}_c_{}_RM_S_L_SIDE_dist_{}.jpg'
#     cv2.imwrite(path_side.format(blockSize, C, distance), rm_s_l_side)

# last_test() # UNCOMMENT to start

In [85]:
qqq = {5:3} # 0 1 1 2 3
test_list = [6,7,8,9]
intersection = [i for i in test_list if i in qqq]
print(intersection)

[]
True
1.4 5.6


In [118]:
import os

def read_images(folder_path):
    if not (folder_path.endswith('/')):
        folder_path += '/'
    print os.listdir(folder_path)
    images = []
    image_names = os.listdir(folder_path)
    for image_name in image_names:
        images.append(cv2.imread(folder_path + image_name, 0)) # `0` --> read images as grayscale
    return image_names, images

# print len(read_images('./images/dataset/newest'))

In [ ]:
def process_images_in_folder(folder_path):
    image_names, images = read_images(folder_path)
    for i in range(len(images)):
        img = images[i]
        img_name = image_names[i]
        
        adaptiveMethod, thresholdType, blockSize, C = cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 35
        
        img_ada = cv2.adaptiveThreshold(img, 255, adaptiveMethod, thresholdType, blockSize, C)
    #     img_ada = cv2.erode(img_ada, np.ones((1,7), dtype=np.int)) # thickens the lines

        runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada) # runs calculation, for thickness and spacing
        line_thickness = calculate_line_thickness(black_runs_flat)
        line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # calculation needs image height

        lines_only_img = get_lines_only_img(img) # ...
        # line_thickness = 3 # CHEAT LINE
        erode_width = int(line_thickness+1) # set to be 4 allways?
        lines_only_img = cv2.erode(lines_only_img, np.ones((erode_width, 1), dtype=np.int)) # thickens the lines

        thickness_mul = 1.5 # staff_thickness_multiplier
    #     thickness_mul = 2 # staff_thickness_multiplier

    #     we can send the grayscale image, also :D  Just send `img` instead of `img_ada`
        result = remove_staff_lines_with_lines_only(img_ada, runs, line_thickness, thickness_mul, lines_only_img)
        result_path = folder_path + img_name + 'RM_S_L_LINES_ONLY_b{}_c{}_ada{}_er_kw{}.jpg'
        cv2.imwrite(result_path.format(blockSize, C, adaptiveMethod, erode_width), result)
    return 0

process_images_in_folder('./images/dataset/newest/')

In [12]:
a = {1:2, -5: 3, 6:2}
sk = a.keys()
sk.sort() # returns None and `sk` becomes [-5, 1, 6]
sk.reverse() # returns None and `sk` becomes [6, 1, -5]

[6, 1, -5]


In [32]:
for ttt in range(0):
    print ttt